[Reference](https://medium.com/swlh/reshape-pandas-data-frames-30c218ff07aa)

In [1]:
import pandas as pd
df = pd.DataFrame({'ID':[1,1,1,1,2,2,3,3,3,4],
                    'Type':['A','B','C','E','D','A','E','B','C','A'],
                     'Value':['L','L','M','H','H','H','L','M','M','M']})

df

,ID,Type,Value
0,1,A,L
1,1,B,L
2,1,C,M
3,1,E,H
4,2,D,H
5,2,A,H
6,3,E,L
7,3,B,M
8,3,C,M
9,4,A,M


In [2]:
# Aggregate the data by ID
df_agg = df.groupby('ID', as_index=False)[['Type','Value']].agg(lambda x: '|'.join(x))
df_agg

,ID,Type,Value
0,1,A|B|C|E,L|L|M|H
1,2,D|A,H|H
2,3,E|B|C,L|M|M
3,4,A,M


In [3]:
df_agg['Type'] = df_agg['Type'].apply(lambda x: x.split("|"))
df_agg['Value'] = df_agg['Value'].apply(lambda x: x.split("|"))

df_agg

,ID,Type,Value
0,1,"[A, B, C, E]","[L, L, M, H]"
1,2,"[D, A]","[H, H]"
2,3,"[E, B, C]","[L, M, M]"
3,4,[A],[M]


In [4]:
df_agg['Type_Value']= df_agg.apply(lambda x: list(zip(x.Type,x.Value)), axis=1)
df_agg

,ID,Type,Value,Type_Value
0,1,"[A, B, C, E]","[L, L, M, H]","[(A, L), (B, L), (C, M), (E, H)]"
1,2,"[D, A]","[H, H]","[(D, H), (A, H)]"
2,3,"[E, B, C]","[L, M, M]","[(E, L), (B, M), (C, M)]"
3,4,[A],[M],"[(A, M)]"


In [5]:
df_agg = df_agg.explode('Type_Value') 
df_agg

,ID,Type,Value,Type_Value
0,1,"[A, B, C, E]","[L, L, M, H]","(A, L)"
0,1,"[A, B, C, E]","[L, L, M, H]","(B, L)"
0,1,"[A, B, C, E]","[L, L, M, H]","(C, M)"
0,1,"[A, B, C, E]","[L, L, M, H]","(E, H)"
1,2,"[D, A]","[H, H]","(D, H)"
1,2,"[D, A]","[H, H]","(A, H)"
2,3,"[E, B, C]","[L, M, M]","(E, L)"
2,3,"[E, B, C]","[L, M, M]","(B, M)"
2,3,"[E, B, C]","[L, M, M]","(C, M)"
3,4,[A],[M],"(A, M)"


In [6]:
df_agg[['New_Type','New_Value']] = pd.DataFrame(df_agg['Type_Value'].tolist(), index=df_agg.index) 
df_agg

,ID,Type,Value,Type_Value,New_Type,New_Value
0,1,"[A, B, C, E]","[L, L, M, H]","(A, L)",A,L
0,1,"[A, B, C, E]","[L, L, M, H]","(B, L)",B,L
0,1,"[A, B, C, E]","[L, L, M, H]","(C, M)",C,M
0,1,"[A, B, C, E]","[L, L, M, H]","(E, H)",E,H
1,2,"[D, A]","[H, H]","(D, H)",D,H
1,2,"[D, A]","[H, H]","(A, H)",A,H
2,3,"[E, B, C]","[L, M, M]","(E, L)",E,L
2,3,"[E, B, C]","[L, M, M]","(B, M)",B,M
2,3,"[E, B, C]","[L, M, M]","(C, M)",C,M
3,4,[A],[M],"(A, M)",A,M


In [7]:
df_agg = df_agg[['ID','New_Type', 'New_Value']].\
rename(columns={"New_Type": "Type", "New_Value": "Value"}).\
reset_index(drop=True)
df_agg

,ID,Type,Value
0,1,A,L
1,1,B,L
2,1,C,M
3,1,E,H
4,2,D,H
5,2,A,H
6,3,E,L
7,3,B,M
8,3,C,M
9,4,A,M


In [8]:
df_agg.equals(df)

True